In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

In [ ]:
ID_COOKIES_BUTTON = "onetrust-accept-btn-handler"
CLASS_ITEM_SALE = "new-item-box__overlay"
KEY_WORD = "Sverige"
DATA_TESTID_LOCATION_PARENT = "item-details-location"
CLASS_LOCATION_VALUE = "details-list__item-value"
CLASS_NEXT_PAGE = "web_ui__Pagination__next"

In [ ]:
driver = webdriver.Chrome()

Aller sur le site de vinted

In [ ]:
driver.get("https://www.vinted.se/")

Accepter les cookies si nécessaire

In [ ]:
try:
    cookies_button = driver.find_element(By.ID, ID_COOKIES_BUTTON)
    cookies_button.click()
except Exception as e:
    # S'il n'y a pas la popup des cookies
    pass 

Sélectionner manuellement les filtres par défaut

On cherche maintenant à récupérer tous les URLS de tous les produits

In [ ]:
def extract_urls_page():
    """
    Extract all URLs of products for sale on the opened page.
    """
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    items = soup.find_all("a", class_=CLASS_ITEM_SALE)
    urls = [item['href'] for item in items]
    return urls

On filtre maintenant les urls en fonction de ce que souhaite l'utilisateur

In [ ]:
def filter_urls_by_location():
    url_product_list = driver.current_url
    urls_all = extract_urls_page()
    urls_filtered = []
    urls_filtered_wrong = []
    for url in urls_all:
        driver.get(url)
        time.sleep(0.5)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        div_location = soup.find("div", {"data-testid": DATA_TESTID_LOCATION_PARENT})

        if div_location:
            location = div_location.find("div", class_=CLASS_LOCATION_VALUE)
            if location:
                if KEY_WORD in location.text:
                    urls_filtered.append(url) 
                else :
                    urls_filtered_wrong.append(url)
            else:
                print("Localisation non mentionnée")
        else:
            print("Description localisation manquante")
            print(driver.page_source)

        # driver.back()  
    driver.get(url_product_list) # Revenir en arrière pour continuer la recherche
    return urls_filtered, urls_filtered_wrong

Parcourir toute les pages

In [ ]:
def go_to_next_page():
    # Passer à la page suivante si elle existe
    try:
        next_button = driver.find_element(By.CLASS_NAME, CLASS_NEXT_PAGE)
        next_button.click()
        WebDriverWait(driver, 10).until(EC.staleness_of(next_button))
        return True
    except Exception as e:
        return False

In [ ]:
urls_totales = []
next_page = True
i = 0
while next_page and i < 10:
    i+=1
    urls_filtered, url_wrong = filter_urls_by_location()
    urls_totales.extend(urls_filtered)
    next_page = go_to_next_page()

In [ ]:
urls_totales

Fermer le navigateur

In [ ]:
driver.quit()